# TRMF: temporal regularized matrix factorization

>**Reference**: Hsiang-Fu Yu, Nikhil Rao, Inderjit S. Dhillon, 2016. [*Temporal regularized matrix factorization for high-dimensional time series prediction*](http://www.cs.utexas.edu/~rofuyu/papers/tr-mf-nips.pdf). 30th Conference on Neural Information Processing Systems (*NIPS 2016*), Barcelona, Spain. [[Matlab code](https://github.com/rofuyu/exp-trmf-nips16)]

## 1. Matrix factorization

### 1.1. Matrix factorization and a novel <span style="color:blue">autoregressive temporal regularizer</span>

Consider a commonly used <span style="color:blue">matrix factorization</span> for completion task:

$$Y=WX^T$$
where $W\in\mathbb{R}^{m\times r},X\in\mathbb{R}^{n\times r}$ are factor matrices decomposed from data matrix $Y\in\mathbb{R}^{m\times n}$. We can also treat matrix $Y$ as <span style="color:blue">a $m$-dimensional time series</span> with <span style="color:blue">$y_{it}$ being the observation at the $t$-th time slot of the $i$-th time series</span>.

In order to model the temporal dependence with time order, we assume that

$$\boldsymbol{x}_{t}\approx\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}$$
where this autoregressive (AR) is specialized by <span style="color:blue">a lag set $\mathcal{L}=\left\{l_1,l_2,...,l_d\right\}$ (e.g., $\mathcal{L}=\left\{1,2,144\right\}$)</span> and <span style="color:blue">weights $\boldsymbol{\theta}_{l}\in\mathbb{R}^{r},\forall l$</span>, and we further define

$$\mathcal{R}_{AR}\left(X\mid \mathcal{L},\Theta,\eta\right)=\frac{1}{2}\sum_{t=l_d+1}^{n}\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)^T\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)+\frac{\eta}{2}\sum_{t=1}^{n}\boldsymbol{x}_{t}^T\boldsymbol{x}_{t}.$$

Thus, <span style="color:blue">TRMF-AR</span> is given by solving

<span style="color:blue">$$\min_{W,X,\Theta}\frac{1}{2}\underbrace{\sum_{(i,t)\in\Omega}\left(y_{it}-\boldsymbol{w}_{i}^T\boldsymbol{x}_{t}\right)^2}_{\text{sum of squared residual errors}}+\lambda_{w}\underbrace{\mathcal{R}_{w}\left(W\right)}_{W-\text{regularizer}}+\lambda_{x}\underbrace{\mathcal{R}_{AR}\left(X\mid \mathcal{L},\Theta,\eta\right)}_{\text{AR-regularizer}}+\lambda_{\theta}\underbrace{\mathcal{R}_{\theta}\left(\Theta\right)}_{\Theta-\text{regularizer}}$$</span>
where $\mathcal{R}_{w}\left(W\right)=\frac{1}{2}\sum_{i=1}^{m}\boldsymbol{w}_{i}^T\boldsymbol{w}_{i}$ and $\mathcal{R}_{\theta}\left(\Theta\right)=\frac{1}{2}\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}^T\boldsymbol{\theta}_{l}$ are regularization terms.

> **Model weakness**: <span style="color:red">weights of regularizers $\left\{\lambda_{w},\lambda_{x},\lambda_{\theta}\right\}$ are not easy to determine!</span>

### 1.2. Learning <span style="color:blue">spatial and temporal embeddings</span>

***Alternating minimization***:

#### 1.2.1. Updates for $W\in\mathbb{R}^{m\times r}$

> Fast algorithms such as <span style="color:blue">alternating least squares or coordinate descent</span> can be applied directly to find $W$.

Solving

<span style="color:blue">$$\min_{W}\frac{1}{2}\underbrace{\sum_{(i,t)\in\Omega}\left(y_{it}-\boldsymbol{w}_{i}^T\boldsymbol{x}_{t}\right)^2}_{\text{sum of squared residual errors}}+\frac{1}{2}\lambda_{w}\underbrace{\sum_{i=1}^{m}\boldsymbol{w}_{i}^T\boldsymbol{w}_{i}}_{\text{sum of squared entries}}$$</span>

by alternative least square for $\boldsymbol{w}_{i},i=1,2,...,m$.

<span style="color:red">$$\boldsymbol{w}_{i}\Leftarrow\left(\sum_{i:(i,t)\in\Omega}\boldsymbol{x}_{t}\boldsymbol{x}_{t}^T+\lambda_{w}I\right)^{-1}\sum_{i:(i,t)\in\Omega}y_{it}\boldsymbol{x}_{t}$$</span>

#### 1.2.2. Updates for $X\in\mathbb{R}^{n\times r}$

Solving

<span style="color:blue">$$\min_{X}\frac{1}{2}\underbrace{\sum_{(i,t)\in\Omega}\left(y_{it}-\boldsymbol{w}_{i}^T\boldsymbol{x}_{t}\right)^2}_{\text{sum of squared residual errors}}
+\frac{1}{2}\lambda_{x}\underbrace{\sum_{t=l_d+1}^{n}\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)^T\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)}_{\text{sum of squared residual errors}}
+\frac{1}{2}\lambda_{x}\eta\underbrace{\sum_{t=1}^{n}\boldsymbol{x}_{t}^T\boldsymbol{x}_{t}}_{\text{sum of squared entries}}$$</span>

- For any $t\in\left\{1,2,...,n\right\}$, we have

$$\underbrace{\sum_{t:(i,t)\in\Omega}\left(y_{it}-\boldsymbol{w}_{i}^{T}\boldsymbol{x}_{t}\right)^2}_{\text{sum of squared residual errors}}=\sum_{t:(i,t)\in\Omega}\left(y_{it}-\boldsymbol{w}_{i}^{T}\boldsymbol{x}_{t}\right)^T\left(y_{it}-\boldsymbol{w}_{i}^{T}\boldsymbol{x}_{t}\right)$$

$$=\boldsymbol{x}_{t}^{T}\left(\sum_{t:(i,t)\in\Omega}\boldsymbol{w}_{i}\boldsymbol{w}_{i}^{T}\right)\boldsymbol{x}_{t}-\boldsymbol{x}_{t}^{T}\left(\sum_{t:(i,t)\in\Omega}y_{it}\boldsymbol{w}_{i}\right)-\left(\sum_{t:(i,t)\in\Omega}y_{it}\boldsymbol{w}_{i}^T\right)\boldsymbol{x}_{t}+\text{const}$$

- For any $t\in\left\{l_d+1,...,n-l_d\right\}$, we have

$$\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)^T\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)+\cdots$$
$$\underbrace{+\left(\boldsymbol{x}_{t+l_d}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t+l_d-l}\right)^T\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t+l_d-l}\right)}_{\text{sum of squared residual errors}}$$

$$=\boldsymbol{x}_{t}^{T}\boldsymbol{x}_{t}-\boldsymbol{x}_{t}^{T}\left(\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)-\left(\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)^T\boldsymbol{x}_{t}
+\sum_{h\in\mathcal{L}}\left(\boldsymbol{\psi}_{t+h}-\boldsymbol{\theta}_{h}\circledast\boldsymbol{x}_{t}\right)^T\left(\boldsymbol{\psi}_{t+h}-\boldsymbol{\theta}_{h}\circledast\boldsymbol{x}_{t}\right)+\text{const}$$

$$=\boldsymbol{x}_{t}^T\left(I+\sum_{h\in\mathcal{L}}\text{diag}(\boldsymbol{\theta}_{h}\circledast\boldsymbol{\theta}_{h})\right)\boldsymbol{x}_{t}-\boldsymbol{x}_{t}^{T}\left(\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}+\sum_{h\in\mathcal{L}}\boldsymbol{\theta}_{h}\circledast\boldsymbol{\psi}_{t+h}\right)+\text{const}$$

where $\boldsymbol{\psi}_{t+h}=\boldsymbol{x}_{t+h}-\sum_{l\in\mathcal{L},l\neq h}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t+h-l}$.

Thus, for <span style="color:red">$t=h_d+1,...,n-h_d$</span>, we can update $\boldsymbol{x}_{t}$ by

<span style="color:red">$$\boldsymbol{x}_{t}\Leftarrow\left(\sum_{t:(i,t)\in\Omega}\boldsymbol{w}_{i}\boldsymbol{w}_{i}^{T}+\left(\lambda_{x}+\lambda_{x}\eta\right)I+\lambda_{x}\sum_{h\in\mathcal{L}}\text{diag}\left(\boldsymbol{\theta}_{h}\circledast\boldsymbol{\theta}_{h}\right)\right)^{-1}\left(\sum_{t:(i,t)\in\Omega}y_{it}\boldsymbol{w}_{i}+\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}+\sum_{h\in\mathcal{L}}\boldsymbol{\theta}_{h}\circledast\boldsymbol{\psi}_{t+h}\right)$$</span>

#### 1.2.3. Updates for $\Theta\in\mathbb{R}^{r\times |\mathcal{L}|}$

Solving the following optimization problem:

<span style="color:blue">$$\min_{\Theta}\frac{1}{2}\lambda_{x}\underbrace{\sum_{t=l_d+1}^{n}\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)^T\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)}_{\text{sum of squared residual errors}}
+\frac{1}{2}\lambda_{\theta}\underbrace{\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}^T\boldsymbol{\theta}_{l}}_{\text{sum of squared entries}}$$</span>

For any $\boldsymbol{\theta}_{h},\forall h\in\mathcal{L}$,

$$\underbrace{\sum_{t=l_d+1}^{n}\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L},l\neq h}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}-\text{diag}(\boldsymbol{x}_{t-h})\boldsymbol{\theta}_{h}\right)^T\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L},l\neq h}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}-\text{diag}(\boldsymbol{x}_{t-h})\boldsymbol{\theta}_{h}\right)}_{\text{sum of squared residual errors}}+\frac{\lambda_{\theta}}{\lambda_{x}}\boldsymbol{\theta}_{h}^{T}\boldsymbol{\theta}_{h}$$

suppose that $\boldsymbol{\pi}_{t}^{(h)}=\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L},l\neq h}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}$, we can update $\boldsymbol{\theta}_{h}$ by

<span style="color:red">$$\boldsymbol{\theta}_{h}\Leftarrow\left(\sum_{t=l_d+1}^{n}\text{diag}(\boldsymbol{x}_{t-h}\circledast\boldsymbol{x}_{t-h})+\frac{\lambda_{\theta}}{\lambda_{x}}I\right)^{-1}\sum_{t=l_d+1}^{n}\text{diag}(\boldsymbol{x}_{t-h})\boldsymbol{\pi}_{t}^{h}$$</span>

## 2. Kernels and implementation

>**Kernel**: Updating $W\in\mathbb{R}^{m\times r}$

$$\boldsymbol{w}_{i}\Leftarrow\left(\sum_{i:(i,t)\in\Omega}\boldsymbol{x}_{t}\boldsymbol{x}_{t}^T+\lambda_{w}I\right)^{-1}\sum_{i:(i,t)\in\Omega}y_{it}\boldsymbol{x}_{t}$$

In [3]:
"""
    update_w(Y,X,lambda_w)
Inputs: Y(m*n), X(m*r), lambda_w(scalar).
"""
function update_w(Y,X,lambda_w)
    W = zeros(size(Y,1),size(X,2))
    for i in [1:size(Y,1)]
        bin = zeros(length(Y[i,:]))
        bin[find(Y[i,:])] = 1
        W[i,:] = inv((bin'*X)'*(bin'*X)+lambda_w)*(Y[i,:]'*X)'
    end
    return W
end

update_w

>**Kernel**: Updating $X\in\mathbb{R}^{n\times r}$

For $t=h_d+1,...,n-h_d$, we can update $\boldsymbol{x}_{t}$ by

$$\boldsymbol{x}_{t}\Leftarrow\left(\sum_{t:(i,t)\in\Omega}\boldsymbol{w}_{i}\boldsymbol{w}_{i}^{T}+\left(\lambda_{x}+\lambda_{x}\eta\right)I+\lambda_{x}\sum_{h\in\mathcal{L}}\text{diag}\left(\boldsymbol{\theta}_{h}\circledast\boldsymbol{\theta}_{h}\right)\right)^{-1}\left(\sum_{t:(i,t)\in\Omega}y_{it}\boldsymbol{w}_{i}+\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}+\sum_{h\in\mathcal{L}}\boldsymbol{\theta}_{h}\circledast\boldsymbol{\psi}_{t+h}\right)$$
where $\boldsymbol{\psi}_{t+h}=\boldsymbol{x}_{t+h}-\sum_{l\in\mathcal{L},l\neq h}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t+h-l}$.

In [6]:
"""
    update_x(Y,W,X,theta,Lags,lambda_x,eta)
Inputs: Y(m*n), W(m*r), X(n*r), theta(d*r), Lags(d*1), lambda_x(scalar), eta(scalar).
"""
function update_x(Y,W,X,theta,Lags,lambda_x,eta)
    r = size(X,2)
    for t in [1:size(X,1)]
        bin = zeros(length(Y[:,t]))
        bin[find(Y[:,t])] = 1
        var1 = (bin'*W)'*(bin'*W)+lambda_x*eta*diagm(ones(r))
        var2 = (Y[:,t]'*W)'
        if t <= max(Lags)
            Pt = zeros(r,r);
            Qt = zeros(r);
        else
            Pt = diagm(ones(r));
            Qt = sum(theta.*X[t-Lags,:],1)';
        end
        if t <= size(X,1)-min(Lags)
            if t > max(Lags) && t <= size(X,1)-max(Lags)
                index = [1:d]
            else
                index = find(x->(x>max(Lags) & x<size(X,1)),t+Lags)
            end
            Mk = zeros(r,r)
            Nk = zeros(r)
            for k in index
                Mk = zeros(r,r)+lambda_x*diagm(theta[k,:].*theta[k,:])
                Nk = zeros(r)+theta[k,:].*(X[t+Lags[k],:]-sum(theta.*X[t+Lags[k]-Lags,:],1)'+theta[k,:].*X[t,:])
            end
            X[t,:] = inv(var1+lambda_x*Pt+lambda_x*Mk)*(var2+Qt+Nk)
        else
            X[t,:] = inv(var1+lambda_x*diagm(ones(r)))*(var2+sum(theta.*X[t-Lags,:],1)')
        end
    end
    return X
end

update_x

> **Kernel**: Updating $\Theta\in\mathbb{R}^{d\times r}$

$$\boldsymbol{\theta}_{h}\Leftarrow\left(\sum_{t=l_d+1}^{n}\text{diag}(\boldsymbol{x}_{t-h}\circledast\boldsymbol{x}_{t-h})+\frac{\lambda_{\theta}}{\lambda_{x}}I\right)^{-1}\sum_{t=l_d+1}^{n}\text{diag}(\boldsymbol{x}_{t-h})\boldsymbol{\pi}_{t}^{h}$$
where $\boldsymbol{\pi}_{t}^{(h)}=\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L},l\neq h}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}$.

In [7]:
"""
    update_theta(X,theta,Lags,lambda_x,lambda_theta)
Inputs: X(n*r), theta(d*r), Lags(d*1), lambda_x(scalar),lambda_theta(scalar).
"""
function update_theta(X,theta,Lags,lambda_x,lambda_theta)
    d = size(theta,1)
    r = size(theta,2)
    for k in [1:d]
        var1 = X[max(Lags)+1-Lags[k]:size(X,1)-Lags[k],:]
        var2 = inv(diagm(sum(var1.*var1,1))+lambda_theta/lambda_x*diagm(ones(r)))
        var3 = zeros(r)
        for t in [max(Lags)+1-Lags[k]:size(X,1)-Lags[k]]
            var3 = zeros(r)+X[t,:].*(X[t+Lags[k],:]-sum(theta.*X[t+Lags[k]-Lags,:],1)'+theta[k,:].*X[t,:])
        end
        theta[k,:] = var2*var3
    end
end

update_theta

## 3. Urban traffic speed data set

## 4. Missing data imputation

## 5. Rolling prediction with incomplete data